In [1]:
import os, sys
project_root = os.path.abspath(os.path.join(os.path.dirname('../')))
sys.path.append(project_root)

from deepface.basemodels import VGGFace
tf_model = VGGFace.loadModel()

import torch
from core.face.vggface import VGGFace as PTVGGFace
torch_model = PTVGGFace()

weights_file = os.path.join(project_root, "pretrained/vggface.pth")
torch_model.load_state_dict(torch.load(weights_file))

2023-06-13 23:19:23.130381: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-13 23:19:23.151319: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 23:19:24.824523: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20548 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9
2023-06-13 23:19:24.824858: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device

<All keys matched successfully>

In [2]:
import cv2
import numpy as np
test_img = '/home/jixian/tmp/face_0_cropped.jpg'
img = cv2.imread(test_img)
img = cv2.resize(img, (224, 224))
tf_img = np.expand_dims(img, axis=0)

tf_res = tf_model.predict(tf_img, verbose=0)[0].tolist()
print(tf_res[:10])
tf_res = tf_model.predict(tf_img, verbose=0)[0].tolist()
print(tf_res[:10])
# convert img to torch tensor

img = cv2.imread(test_img)
img = cv2.resize(img, (224, 224))
img = img.astype(np.float32)
img = img.transpose(2, 0, 1)
torch_img = torch.from_numpy(img)

torch_model.eval()
with torch.no_grad():  
  torch_res = torch_model(torch_img).reshape(-1).detach().numpy().tolist()
  print(torch_res[:10])

  features = torch_model.features(torch_img)  
  torch_res2 = torch_model.embedding(features).detach().numpy().tolist()
  print(torch_res2)

2023-06-13 23:19:28.922837: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902
2023-06-13 23:19:30.148522: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


[2.270510196685791, -0.28317683935165405, 0.6137429475784302, 1.8658781051635742, 2.337536573410034, 3.0819878578186035, 2.980890989303589, 1.5023161172866821, 2.195514678955078, 0.2994230389595032]
[2.270510196685791, -0.28317683935165405, 0.6137429475784302, 1.8658781051635742, 2.337536573410034, 3.0819878578186035, 2.980890989303589, 1.5023161172866821, 2.195514678955078, 0.2994230389595032]
[2.2696855068206787, -0.28304508328437805, 0.613831639289856, 1.8660333156585693, 2.337951183319092, 3.081942558288574, 2.98045015335083, 1.502403974533081, 2.1956019401550293, 0.29918134212493896]
[2.2696855068206787, -0.28304508328437805, 0.613831639289856, 1.8660333156585693, 2.337951183319092, 3.081942558288574, 2.98045015335083, 1.502403974533081, 2.1956019401550293, 0.29918134212493896, 2.9592957496643066, -1.6453789472579956, 0.8032979965209961, 1.0355509519577026, -0.6896162033081055, -0.6283295154571533, 1.8462504148483276, 0.9717873334884644, 2.6028153896331787, 7.380914688110352, -0.7

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity([tf_res], [torch_res]))
print(cosine_similarity([tf_res], [torch_res2]))
print(cosine_similarity([torch_res], [torch_res2]))


[[0.99999999]]
[[0.99999999]]
[[1.]]
